---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import numpy as np
import pandas as pd
from functools import reduce
from string import digits

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [2]:
# Load energy data from 'Energy Indicators.xls':
energy = pd.read_excel(
    header=None,
    io='Energy Indicators.xls',
    names=[
        'Country',
        'Energy Supply',
        'Energy Supply per Capita',
        '% Renewable'
    ],
    skipfooter=38,
    skiprows=18,
    usecols=list(range(2, 6))
)

# Transform the 'energy' DataFrame:

# Deal with non-numeric data in 'Energy Supply' and 'Energy Supply per Capita' columns:
tmp_cols = ['Energy Supply', 'Energy Supply per Capita']
energy.loc[:, tmp_cols] = energy.loc[:, tmp_cols].replace('...', np.nan)

# Convert 'Energy Supply' to GJ:
energy['Energy Supply'] = energy['Energy Supply'].mul(1E6, fill_value=np.nan)

# Rename countries:
new_names = {
    'China, Hong Kong Special Administrative Region': 'Hong Kong',
    'Republic of Korea': 'South Korea',
    'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
    'United States of America': 'United States'
}
energy['Country'] = (
    energy['Country'].str.translate(str.maketrans('', '', digits))
    .apply(lambda t: new_names[t] if t in new_names else t)
    .apply(lambda t: t.split(' (')[0])
)

# Set 'Country' as the index (useful for merging the DataFrames later):
energy.set_index('Country', inplace=True)

In [3]:
# Load GDP data from 'world_bank.csv':
GDP = pd.read_csv('world_bank.csv', skiprows=4)

# Transform the 'GDP' DataFrame:

# Rename the column containing the countries:
GDP.rename(columns={'Country Name': 'Country'}, inplace=True)

# Rename countries:
new_names = {
    'Hong Kong SAR, China': 'Hong Kong',
    'Iran, Islamic Rep.': 'Iran',
    'Korea, Rep.': 'South Korea'
}
GDP['Country'] = GDP['Country'].apply(lambda t: new_names[t] if t in new_names else t)

# Set 'Country' as the index (useful for merging the DataFrames later):
GDP.set_index('Country', inplace=True)

In [4]:
# Load data from 'scimagojr-3.xlsx':
ScimEn = pd.read_excel(io='scimagojr-3.xlsx', index_col=1)

In [5]:
def answer_one():
    """This function does the merging."""

    # List of DataFrames we want to merge:
    df_to_merge = [
        ScimEn[ScimEn['Rank'] < 16],
        energy,
        GDP.loc[:, [str(year) for year in range(2006, 2016)]]
    ]

    # Actual merging:
    df = reduce(
        lambda l, r: pd.merge(l, r, how='inner', left_index=True, right_index=True),
        df_to_merge
    )
    return df


Top15 = answer_one()

### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [6]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [7]:
def answer_two():

    # List of DataFrames we want to merge:
    df_to_merge = [
        ScimEn,
        energy,
        GDP.loc[:, [str(year) for year in range(2006, 2016)]]
    ]

    # Union of the DataFrames:
    union = reduce(
        lambda l, r: pd.merge(l, r, how='outer', left_index=True, right_index=True),
        df_to_merge
    )

    # Intersection of the DataFrames:
    intersection = reduce(
        lambda l, r: pd.merge(l, r, how='inner', left_index=True, right_index=True),
        df_to_merge
    )

    return len(union) - len(intersection)


answer_two()

156

## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [8]:
def answer_three():
    years = [str(year) for year in range(2006, 2016)]
    tmp_series = Top15.loc[:, years].mean(axis=1).sort_values(ascending=False)
    tmp_series.name = 'avgGDP'
    return tmp_series


avgGDP = answer_three()

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [9]:
def answer_four():
    idx = avgGDP.index.values[5]
    return Top15.loc[idx, '2015'] - Top15.loc[idx, '2006']


answer_four()

246702696075.3999

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [10]:
def answer_five():
    return Top15['Energy Supply per Capita'].mean()


answer_five()

157.59999999999999

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [11]:
def answer_six():
    country = Top15['% Renewable'].idxmax()
    percentage = Top15['% Renewable'].max()
    return (country, percentage)


answer_six()

('Brazil', 69.648030000000006)

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [12]:
def answer_seven():
    ratios = Top15['Self-citations'].div(Top15['Citations'])
    country = ratios.idxmax()
    ratio = ratios.max()
    return (country, ratio)


answer_seven()

('China', 0.68931261793894216)

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [13]:
def answer_eight():
    Top15['PopEst'] = Top15['Energy Supply'].div(Top15['Energy Supply per Capita'])
    return Top15['PopEst'].nlargest(n=3).idxmin()


answer_eight()

'United States'

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [14]:
def answer_nine():
    Top15['Citable docs per Capita'] = Top15['Citable documents'].div(Top15['PopEst'])
    return Top15['Citable docs per Capita'].corr(Top15['Energy Supply per Capita'], method='pearson')


answer_nine()

0.79400104354429457

In [15]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    Top15.plot(
        x='Citable docs per Capita',
        y='Energy Supply per Capita',
        kind='scatter',
        xlim=[0, 0.0006]
    )

In [16]:
#plot9()

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [17]:
def answer_ten():
    median_percentage = Top15['% Renewable'].median()
    tmp_series = (Top15.sort_values('Rank')['% Renewable'] >= median_percentage).astype('int64')
    tmp_series.name = 'HighRenew'
    return tmp_series


answer_ten()

Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: HighRenew, dtype: int64

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [18]:
ContinentDict = {
    'Australia': 'Australia',
    'Brazil': 'South America',
    'Canada': 'North America',
    'China': 'Asia',
    'France': 'Europe',
    'Germany': 'Europe',
    'India': 'Asia',
    'Iran': 'Asia',
    'Italy': 'Europe',
    'Japan': 'Asia',
    'Russian Federation': 'Europe',
    'South Korea': 'Asia',
    'Spain': 'Europe',
    'United Kingdom': 'Europe',
    'United States': 'North America'
}

In [19]:
def answer_eleven():
    FunctionDict = {
        'count': 'size',
        'sum': 'sum',
        'mean': 'mean',
        'std': 'std'
    }
    tmp_df = (
        Top15.groupby(ContinentDict).PopEst
        .agg(list(FunctionDict.keys())).rename(columns=FunctionDict)
    )
    # This conversion is unneccessary, but the autograder complains if the results are integers.
    tmp_df['size'] = tmp_df['size'].astype('float64')
    tmp_df.index.name = 'Continent'
    return tmp_df


answer_eleven()

,size,sum,mean,std
Continent,,,,
Asia,5.0,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1.0,2.331602e+07,2.331602e+07,NaN
Europe,6.0,4.579297e+08,7.632161e+07,3.464767e+07
North America,2.0,3.528552e+08,1.764276e+08,1.996696e+08
South America,1.0,2.059153e+08,2.059153e+08,NaN


### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [20]:
def answer_twelve():
    tmp_df = Top15.reset_index()
    tmp_df['Continent'] = tmp_df['Country'].map(ContinentDict)
    tmp_df['% Renewable'] = pd.cut(tmp_df['% Renewable'], bins=5)
    tmp_df.set_index(['Continent', '% Renewable'], inplace=True)
    tmp_series = tmp_df.Country.groupby(level=['Continent', '% Renewable']).count()
    tmp_series.name = 'Count'
    return tmp_series


answer_twelve()

Continent      % Renewable     
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
Australia      (2.212, 15.753]     1
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
North America  (2.212, 15.753]     1
               (56.174, 69.648]    1
South America  (56.174, 69.648]    1
Name: Count, dtype: int64

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [21]:
def answer_thirteen():
    tmp_series = Top15['PopEst'].apply(lambda t: '{:,}'.format(t))
    tmp_series.name = 'PopEst'
    return tmp_series


answer_thirteen()

Country
Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
Canada                  35,239,864.86486486
China                 1,367,645,161.2903225
Germany                 80,369,696.96969697
Spain                    46,443,396.2264151
France                  63,837,349.39759036
United Kingdom         63,870,967.741935484
India                 1,276,730,769.2307692
Iran                    77,075,630.25210084
Italy                  59,908,256.880733944
Japan                  127,409,395.97315437
South Korea            49,805,429.864253394
Russian Federation            143,500,000.0
United States          317,615,384.61538464
Name: PopEst, dtype: object

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [ ]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [ ]:
#plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!